# Experimento TCC

## Libs

In [1]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from psutil import cpu_count
import pandas as pd
import numpy as np

In [2]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient
from modules.utils import folds_run, sklearn_folds_run

%load_ext autoreload
%autoreload 2

In [3]:
dtypes = {
    'tau1': float, 'tau2': float, 'tau3': float, 'tau4': float,
    'p1': float, 'p2': float,'p3': float, 'p4': float, 
    'g1': float, 'g2': float, 'g3': float, 'g4': float}

EletricalGrid = pd.read_csv('dataset/Data_for_UCI_named.csv',
                            dtype=dtypes, nrows=300).drop('stab', axis=1)

In [4]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [20]:
my_pso_options = {
    'optimization_func':PSO, 
    'max_iter':10, 'k':5, 
    'cpu_count':cpu_count(),
    'my_pso':True,
    'swarm_size':200
}

not_my_pso_options = {
    'optimization_func':PSO, 
    'max_iter':10, 'k':5, 
    'cpu_count':1,
    'my_pso':True,
    'swarm_size':200
}

In [ ]:
results = folds_run(X, y, model_class=NCA, model_options=my_pso_options)
for i in results:
    print(f'{i} mean = {np.array(results[i]).mean()}')

0it [00:00, ?it/s]

[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  20 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  32 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  36 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  48 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done  52 tasks      | elapsed:    1.1s

1it [00:35, 35.82s/it]

[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  20 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  32 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  36 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  48 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done  52 tasks      | elapsed:    1.1s

2it [01:11, 35.95s/it]

[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  20 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  32 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  36 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  48 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  52 tasks      | elapsed:    1.1s

3it [01:48, 36.21s/it]

[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  20 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  28 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  32 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  36 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  48 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  52 tasks      | elapsed:    1.0s

In [18]:
results = folds_run(X, y, model_class=NCA, model_options=not_my_pso_options)
for i in results:
    print(f'{i} mean = {np.array(results[i]).mean()}')

0it [00:00, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

1it [00:13, 13.27s/it]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

2it [00:26, 13.25s/it]

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

4it [00:53, 13.50s/it]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

5it [01:07, 13.53s/it]

acc mean = 0.8066666666666666
mcc mean = 0.5708847073790848


In [8]:
#nca_sklearn = NeighborhoodComponentsAnalysis()
#sklearn_options = {'n_components':None, 'max_iter':1000,'random_state':42}

In [9]:
#npX, npy = np.asarray(X), np.asarray(y)
#results = sklearn_folds_run(npX, npy, 100, model_class=NeighborhoodComponentsAnalysis, model_options=sklearn_options)
#for i in results:
#    print(f'{i} mean = {np.array(results[i]).mean()}')